# HOME ASSIGNMENT #3: SLACK API - TO GSHEET
**Mục đích của bài Assignment**
- Lấy thông tin các Users từ Slack của DataCracy (BTC, Mentors và Learners)
- `**[Optional 1]**` Đưa danh sách Users lên Google Spreadsheet, để theo dõi 
- `**[Optional 2]**` Lấy thông tin Assignment Submission và số Reviews trên `#atom-assignmentnt2` và cập nhật lên Spreadsheet, để theo dõi các học viên đã nộp bài và được review

**Các kiến thức sẽ áp dụng**
- Ôn lại và luyện tập thêm về concept API (cụ thể sử dụng API Slack)
- Trích xuất thông tin từ JSON
- Dùng module gspread để đưa thông tin lên Google Spreadsheet

## 0. Load Modules

In [1]:
import requests #-> Để gọi API
import re #-> Để xử lý data dạng string
from datetime import datetime as dt #-> Để xử lý data dạng datetime
import gspread #-> Để update data lên Google Spreadsheet
from gspread_dataframe import set_with_dataframe #-> Để update data lên Google Spreadsheet
import pandas as pd #-> Để update data dạng bản
import json 
from oauth2client.service_account import ServiceAccountCredentials #-> Để nhập Google Spreadsheet Credentials
import os

## 1. Slack API: User List
* Bạn có thể đọc lại về concept API [HERE](https://anhdang.gitbook.io/datacracy/atom/3-data-tools-2/3.2-spotify-api-and-postman)
* Assignment này sẽ dùng Slack API để lấy thông tin về Learners và theo dõi các bài tập đã nộp và được review (sau đó cập nhật lên Google Spreadsheet)
* ===> **NOTICE**: Slack API authorize bằng Bearer Token `xoxb-...-...-...` (Sẽ được cung cấp riêng)
* Update file `env_variable.json` như trong [Assignment#2](../assignment_2/home_assignment_2.ipynb)
* ==> Nếu bạn dùng Google Colab, upload file vào Colab ([Hướng dẫn](https://colab.research.google.com/notebooks/io.ipynb))

In [2]:
!ls

env_variables.json  home_assignment_3.ipynb  README.md	slack-bot-key.json


In [4]:
with open('env_variables.json', 'r') as j:
    json_data = json.load(j)

In [5]:
## Load SLACK_BEARER_TOKEN
os.environ['SLACK_BEARER_TOKEN'] = json_data['SLACK_BEARER_TOKEN'] 


In [7]:
## Gọi API từ Endpoints (Input - Token được đưa vào Headers)
## Challenge: Thử gọi API này bằng Postman
endpoint = "https://slack.com/api/users.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response_json = requests.post(endpoint, headers=headers).json() 
user_dat = response_json['members']

[{'id': 'USLACKBOT', 'team_id': 'T01B7SGGMLJ', 'name': 'slackbot', 'deleted': False, 'color': '757575', 'real_name': 'Slackbot', 'tz': 'America/Los_Angeles', 'tz_label': 'Pacific Daylight Time', 'tz_offset': -25200, 'profile': {'title': '', 'phone': '', 'skype': '', 'real_name': 'Slackbot', 'real_name_normalized': 'Slackbot', 'display_name': 'Slackbot', 'display_name_normalized': 'Slackbot', 'fields': None, 'status_text': '', 'status_emoji': '', 'status_expiration': 0, 'avatar_hash': 'sv41d8cd98f0', 'always_active': True, 'first_name': 'slackbot', 'last_name': '', 'image_24': 'https://a.slack-edge.com/80588/img/slackbot_24.png', 'image_32': 'https://a.slack-edge.com/80588/img/slackbot_32.png', 'image_48': 'https://a.slack-edge.com/80588/img/slackbot_48.png', 'image_72': 'https://a.slack-edge.com/80588/img/slackbot_72.png', 'image_192': 'https://a.slack-edge.com/80588/marketing/img/avatars/slackbot/avatar-slackbot.png', 'image_512': 'https://a.slack-edge.com/80588/img/slackbot_512.png',

### TODO #1
Hoàn tất đoạn code sau

In [9]:
## Loop qua JSON file và extract các thông tin quan trọng (id, name, display_name, real_name_normalized, title, phone, is_bot)
## Hint: Bạn có thể dùng Postman hoặc in user_dat JSON để xem cấu trúc (schema), dùng Ctrl+F để tìm các keys (id, name, display_name, real_name_normalized, title, phone, is_bot)
user_dict = {'user_id':[], 'name':[], 'display_name':[],'real_name':[],'title':[],'phone':[],'is_bot':[]}
for i in range(len(user_dat)):
  user_dict['user_id'].append(user_dat[i]['id'])
  user_dict['name'].append(user_dat[i]['name'])
  user_dict['display_name'].append(user_dat[i]['profile']['display_name'])
  user_dict['real_name'].append(user_dat[i]['profile']['real_name_normalized'])
  user_dict['title'].append(user_dat[i]['profile']['title'])
  user_dict['phone'].append(user_dat[i]['profile']['phone'])
  user_dict['is_bot'].append(user_dat[i]['is_bot'])


In [10]:
user_df = pd.DataFrame(user_dict) ## Dùng pandas để convert dictionaries thành bảng

user_df.head(5) ## Chỉ in 5 dòng đầu (chủ yếu để xem cấu trúc)

,user_id,name,display_name,real_name,title,phone,is_bot
0,USLACKBOT,slackbot,Slackbot,Slackbot,,,False
1,U01AT4T75JB,loclexuan26392,Loc Le Xuan,Loc Le Xuan,,,False
2,U01AVDY7JET,locle.ds,Loc Le Xuan,Loc Le Xuan,,,False
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False
4,U01C48T7S1J,huyenhoang.design,Thanh Huyen Hoang,Thanh Huyen Hoang,,,False


In [11]:
user_df[user_df.display_name == 'MAD'] ## Lọc thông tin của MAD, trên DataFrame (bạn có thể Google thêm)

,user_id,name,display_name,real_name,title,phone,is_bot
3,U01BE2PR6LU,maianhdang.ftu,MAD,Dặng Huỳnh Mai Anh,Technical Contents,,False


-------------- HẾT PHẦN BẮT BUỘC ---------------------

## Option 1: Update data => Google SpreadSheet

### TODO#2
Tạo service account (output là file json), file này để cho phép ta access vào Google Spreadsheet:

1. Làm theo hướng dẫn: [Google Create a Service Account](https://support.google.com/a/answer/7378726?hl=en)
![google_service_account](../img/google_service_account.png)
2. Lưu file JSON (chứa credential về máy)
![gservice_acc_json](../img/gservice_acc_json.png)
3. Nhớ Enable [Google Drive API](https://console.cloud.google.com/marketplace/product/google/drive.googleapis.com?q=search&referrer=search&project=quickstart-313303) (Nếu bạn chạy code báo lỗi chưa enable API thì vào link trong phần lỗi để Enable, sau khi kích hoạt có thể cần vài phút để chạy được)
![enable_api](../img/enable_api.png)
* ==> Upload file Gsheet Credential JSON nếu bạn dùng Colab 
* ==> Nếu bạn để key trong repo git, **NHỚ** để file json vào `.gitignore` để không bị leaked key)


In [12]:
!ls

env_variables.json  home_assignment_3.ipynb  README.md	slack-bot-key.json


In [13]:
## Authorize bằng JSON
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    'slack-bot-key.json', scope)
gc = gspread.authorize(credentials)
print("DONE!")

DONE!


**Tạo Spreadsheet**

1. Tạo Spreadsheet trên google
2. Invite account trong `client_email` (file JSON Gsheet Credential bên trên) vào Spreadsheet (quyền Editor)
![enable_api](../img/enable_api.png)
3. Lấy `SPREADSHEET_KEY` (nằm trong chính URL của Spreadhstee): `https://docs.google.com/spreadsheets/d/<SPREADSHEET_KEY>/edit#gid=0`

![add_gsheet](../img/add_gsheet.png)

In [14]:
# ACCES GOOGLE SHEET
sheet_index_no = 0
spreadsheet_key = '1mrHOuXzCOtgsIPQcm-pg8VK6uE3DM91u-RGXO_Vfexs' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 

# APPEND DATA TO SHEET
set_with_dataframe(worksheet, user_df) #-> Upload user_df vào Sheet đầu tiên trong Spreadsheet

# DONE: Bây giờ bạn có thể mở spreadsheet và kiểm tra nội dung đã update chứ

![slack_user_gsheet](../img/slack_user_gsheet.png)

-------------- HẾT PHẦN OPTION 1 ---------------------

## Option 2: Ai đã nộp bài?


### Slack API: Channel List

In [15]:
## Gọi SLACK API để list tất cả các channel
endpoint = "https://slack.com/api/conversations.list"
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}
response = requests.post(endpoint, headers=headers).json() 

channel_ls = response['channels']

In [16]:
channel_ls[0] ## Thử extract record đầu tiên để xem schema  => name: general, id: C01B4PVGLVB

{'id': 'C01B4PVGLVB',
 'name': 'general',
 'is_channel': True,
 'is_group': False,
 'is_im': False,
 'created': 1600856703,
 'is_archived': False,
 'is_general': True,
 'unlinked': 0,
 'name_normalized': 'general',
 'is_shared': False,
 'parent_conversation': None,
 'creator': 'U01BE2PR6LU',
 'is_ext_shared': False,
 'is_org_shared': False,
 'shared_team_ids': ['T01B7SGGMLJ'],
 'pending_shared': [],
 'pending_connected_team_ids': [],
 'is_pending_ext_shared': False,
 'is_member': False,
 'is_private': False,
 'is_mpim': False,
 'topic': {'value': '', 'creator': '', 'last_set': 0},
 'purpose': {'value': 'This is the one channel that will always include everyone. It’s a great spot for announcements and team-wide conversations.',
  'creator': 'U01BE2PR6LU',
  'last_set': 1600856703},
 'previous_names': [],
 'num_members': 65}

In [17]:
#TODO 3
for i in channel_ls:
    if i['name'] == 'atom-assignment2':
        print(i)

{'id': 'C021FSDN7LJ', 'name': 'atom-assignment2', 'is_channel': True, 'is_group': False, 'is_im': False, 'created': 1620677142, 'is_archived': False, 'is_general': False, 'unlinked': 0, 'name_normalized': 'atom-assignment2', 'is_shared': False, 'parent_conversation': None, 'creator': 'U01BE2PR6LU', 'is_ext_shared': False, 'is_org_shared': False, 'shared_team_ids': ['T01B7SGGMLJ'], 'pending_shared': [], 'pending_connected_team_ids': [], 'is_pending_ext_shared': False, 'is_member': True, 'is_private': False, 'is_mpim': False, 'topic': {'value': 'Assigment#2 - Mailbot, API, Web Scrapping', 'creator': 'U01BE2PR6LU', 'last_set': 1621074502}, 'purpose': {'value': 'Where learners submit assignments and give review for others', 'creator': 'U01BE2PR6LU', 'last_set': 1621074519}, 'previous_names': [], 'num_members': 40}


### TODO#3 
* Tìm id của channel #atom-assignment2

### Slack API: List messages trong 1 channel

In [18]:
endpoint = "https://slack.com/api/conversations.history"
data = {"channel": "C021FSDN7LJ"} ## This is ID of assignment#1 channel
headers = {"Authorization": "Bearer {}".format(os.environ['SLACK_BEARER_TOKEN'])}

In [19]:
response_json = requests.post(endpoint, data=data, headers=headers).json()
msg_ls = response_json['messages']

In [20]:
msg_ls[0]

{'type': 'message',
 'subtype': 'channel_join',
 'ts': '1622694679.000700',
 'user': 'U0226MQ73MJ',
 'text': '<@U0226MQ73MJ> has joined the channel',
 'inviter': 'U01BE2PR6LU'}

In [21]:
not_learners_id = ['U01BE2PR6LU']

In [22]:
## Summarize all submitted assignments + reviews cnt
not_learners_id = ['U01BE2PR6LU'] # -> Remove MA from the user_id
msg_lk = []

msg_dict = {'github': [], 'reply_count': [], 'reply_users_count': [], 'reply_users': [], 'latest_reply':[] } 
for i in range(len(msg_ls)):
  ts = dt.fromtimestamp(float(msg_ls[i]['ts'])) # -> Convert timestamp Epoch thành dàng dễ đọc
  user = msg_ls[i]['user'] # -> Lấy thông tin người post messages
  if msg_ls[i]['user'] not in not_learners_id:
    if 'attachments' in msg_ls[i].keys():
      text = msg_ls[i]['text']
      github_link = re.findall('(?:https?://)?(?:www[.])?github[.]com/[\w-]+/?', text) #-> Submission là các message có link github
      print(github_link)
      if len(github_link) > 0: 
        msg_dict['github'].append(github_link[0])

        if 'reply_count' in msg_ls[i].keys(): 
            msg_dict['reply_count'].append(msg_ls[i]['reply_count']) 
        else: msg_dict['reply_count'].append('') #-> Extract số review
        if 'reply_users_count' in msg_ls[i].keys(): 
            msg_dict['reply_users_count'].append(msg_ls[i]['reply_users_count'])
        else: msg_dict['reply_users_count'].append('')
        if 'reply_users' in msg_ls[i].keys(): 
            msg_dict['reply_users'].append(msg_ls[i]['reply_users'])
        else: msg_dict['reply_users'].append('')
        if 'latest_reply' in msg_ls[i].keys(): 
            msg_dict['latest_reply'].append(dt.fromtimestamp(float(msg_ls[i]['latest_reply'])))
        else: msg_dict['latest_reply'].append('')


msg_df = pd.DataFrame(msg_dict)
msg_df

['https://github.com/nguyenhonghanh2511/']
['https://github.com/nguyenhonghanh2511/', 'https://github.com/nguyenhonghanh2511/']
['https://github.com/cnhhoang850/']
['https://github.com/lethuthao1368/']
['https://github.com/hoaintp/']
['https://github.com/annaho124/']
[]
['https://github.com/chuviethongfpt/']
[]


,github,reply_count,reply_users_count,reply_users,latest_reply
0,https://github.com/nguyenhonghanh2511/,1,1,[U01UTGRMGEQ],2021-05-22 21:18:58.019900
1,https://github.com/nguyenhonghanh2511/,2,2,"[U01UJ9MUADT, U01UTGRMGEQ]",2021-05-23 10:49:36.022700
2,https://github.com/cnhhoang850/,1,1,[U01UMJ5AQ0K],2021-05-22 08:42:07.009000
3,https://github.com/lethuthao1368/,2,2,"[U01V00JK9LH, U01V082PLN5]",2021-05-22 13:20:45.016200
4,https://github.com/hoaintp/,2,2,"[U01UJ9PA0MB, U01UTGVPE7N]",2021-05-25 01:10:16.027500
5,https://github.com/annaho124/,4,3,"[U01UMJ451V1, U01UMC0N5U2, U01UMJ36QD9]",2021-05-21 21:25:32.007200
6,https://github.com/chuviethongfpt/,9,3,"[U01UMJ36QD9, U01UMC0N5U2, U01UMBW8G8J]",2021-05-22 19:30:18.017600


### TODO#4

* Tạo thành 1 bảng chứa các thông tin trên và update lên Spreadsheet (Sheet: Assignment#2 Submission)

In [23]:
sheet_index_no = 1
spreadsheet_key = '1mrHOuXzCOtgsIPQcm-pg8VK6uE3DM91u-RGXO_Vfexs' # input SPREADSHEET_KEY HERE
sh = gc.open_by_key(spreadsheet_key)
worksheet2 = sh.get_worksheet(sheet_index_no) #-> 0 - first sheet, 1 - second sheet etc. 
set_with_dataframe(worksheet2, msg_df)


-------------- HẾT PHẦN OPTION 2 ---------------------